In [57]:
import spikeinterface.core as sc
import spikeinterface.extractors as se

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

#######################################################################
animal_ID = '98'
session_ID = '2024-04-16T10-37-23'

base_folder = r"W:\projects\FlexiVexi\behavioural_data"
session_folder = os.path.join(base_folder, animal_ID, session_ID)
#######################################################################

# Path to raw binary file from Open Ephys within session folder
path_to_recording = r"W:\projects\FlexiVexi\behavioural_data\98\2024-04-16T10-37-23\Open-Ephys\98_2024-04-16_11-37-19"

# Path to Kilosort3 output files within session folder
kilosort_folder = os.path.join(session_folder, 'kilosort3')


**Read in recording and spike sorting**

In [62]:
# Get output from spike sorting using Kilosort3
sorting_KS = se.read_kilosort(folder_path=kilosort_folder)
sorting_KS

# Get recording from open ephys
recording = se.read_openephys(folder_path=path_to_recording, stream_name = 'Record Node 102#Neuropix-PXI-100.ProbeA')
recording

# Create a sorting analyzer
sc.create_sorting_analyzer(sorting_KS, recording)

AttributeError: module 'spikeinterface.core' has no attribute 'create_sorting_analyzer'